In [ ]:
# TODO 성능상승의 GAP을 그래프로 나타내보기.

In [28]:
import pandas as pd
import re

def parse_log_file(log_path):
    data = []
    fold = None
    hyperparams = {}
    capturing_hyperparams = False
    epoch_counter = 0  # epoch 추적을 위한 변수

    with open(log_path, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith("[Outer Fold"):
                # Outer Fold 시작 감지
                match = re.match(r"\[Outer Fold (\d+)\]", line)
                if match:
                    fold = int(match.group(1))
                    epoch_counter = 0  # 새로운 Fold 시작 시 epoch 초기화
            
            elif line.startswith("Hyperparameters:"):
                # 하이퍼파라미터 시작 감지
                hyperparams = {}
                capturing_hyperparams = True

            elif capturing_hyperparams and ": " in line:
                # 하이퍼파라미터 추출
                if line.startswith("EN_model"):
                    capturing_hyperparams = False  # 모델 부분을 건너뛰기
                    continue
                
                key, value = map(str.strip, line.split(": ", 1))
                if key == "batch_size":
                    value = eval(value)  # 문자열 리스트를 실제 리스트로 변환
                elif re.match(r"^\d+(\.\d+)?$", value):  # 숫자 값 검증
                    value = float(value) if "." in value else int(value)
                
                # 특정 불필요한 키 제외
                if key not in ["TX_drug_feat_data_test", "TX_drug_adj_data_test", "TX_gexpr_data_test"]:
                    hyperparams[key] = value

            elif "-auc_val" in line or "-auc_TEST" in line:
                # AUC 결과 파싱
                result, metric = map(str.strip, line.split(" -"))
                row = {
                    "fold": fold,
                    "epoch": epoch_counter,  # 현재 epoch 추가
                    "metric": metric,
                    "value": float(result),
                    **hyperparams  # 하이퍼파라미터 추가
                }
                data.append(row)
                epoch_counter += 1  # epoch 증가
            
            elif line.startswith("Best Params"):
                # 최적 파라미터 및 AUC 추출
                match = re.match(r".*Best Params: (.+), Test AUC: (.+)", line)
                if match:
                    best_params = eval(match.group(1))
                    test_auc = float(match.group(2))
                    row = {
                        "fold": fold,
                        "epoch": epoch_counter,
                        "metric": "best_auc_TEST",
                        "value": test_auc,
                        **best_params  # 최적 파라미터 추가
                    }
                    data.append(row)

    return pd.DataFrame(data)

# 로그 파일 파싱 실행
log_path = './log_FC.txt'  # 실제 로그 파일 경로
parsed_df = parse_log_file(log_path)

# batch_size를 튜플로 변환하여 그룹화 가능하도록 설정
parsed_df['batch_size'] = parsed_df['batch_size'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# 'auc_TEST' 만 필터링
test_only_df = parsed_df[parsed_df['metric'] == 'auc_TEST']

# Early stopping 조건 적용: 10 epoch 이내에 종료된 경우만 선택
early_stopped_df = test_only_df[test_only_df['epoch'] <= 10]

# 'auc_TEST' 기준으로 최고값과 해당 하이퍼파라미터 찾기
max_values_early_stopping = early_stopped_df.loc[
    early_stopped_df.groupby(['fold', 'nz', 'd_dim', 'lr', 'lr_critic', 'lam', 'batch_size'])['value'].idxmax()
]

# 결과를 CSV 파일로 저장
max_values_early_stopping.to_csv("max_hyperparameter_values_auc_TEST_early_stopping.csv", index=False)

# 결과 출력
print(max_values_early_stopping)


   fold  epoch    metric     value   nz  d_dim     lr  lr_critic   lam  \
9     0      9  auc_TEST  0.678362  100    256  0.001        0.1  0.01   

  batch_size  
9  (256, 28)  


In [49]:
# AUC 증가 커브 Plot 그리기
import pandas as pd
import re
import matplotlib.pyplot as plt
def parse_log_file(log_path):
    data = []
    outer_fold = None
    inner_fold = None
    hyperparams = {}
    capturing_hyperparams = False

    with open(log_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Outer Fold 시작 감지
            if line.startswith("[Outer Fold"):
                match = re.match(r"\[Outer Fold (\d+)\]", line)
                if match:
                    outer_fold = int(match.group(1))

            # Inner Fold 시작 감지
            elif line.startswith("[Inner Fold"):
                match = re.match(r"\[Inner Fold (\d+)\]", line)
                if match:
                    inner_fold = int(match.group(1))

            # 하이퍼파라미터 추출 시작
            elif line.startswith("Hyperparameters:"):
                hyperparams = {}
                capturing_hyperparams = True

            # 하이퍼파라미터 값 추출
            elif capturing_hyperparams and ": " in line:
                if line.startswith("EN_model"):
                    capturing_hyperparams = False  # 모델 부분을 건너뛰기
                    continue

                key, value = map(str.strip, line.split(": ", 1))
                if key == "batch_size":
                    value = eval(value)  # 문자열 리스트를 리스트로 변환
                elif re.match(r"^\d+(\.\d+)?$", value):  # 숫자 여부 확인
                    value = float(value) if "." in value else int(value)

                # 특정 불필요한 키 제외
                if key not in ["TX_drug_feat_data_test", "TX_drug_adj_data_test", "TX_gexpr_data_test"]:
                    hyperparams[key] = value

            # AUC 값 추출
            elif "-auc_val" in line or "-auc_TEST" in line:
                result, metric = map(str.strip, line.split(" -"))
                row = {
                    "outer_fold": outer_fold,
                    "inner_fold": inner_fold,
                    "metric": metric,
                    "value": float(result),
                    **hyperparams  # 하이퍼파라미터 추가
                }
                data.append(row)

            # 최적 하이퍼파라미터와 테스트 AUC 추출
            elif line.startswith("Best Params"):
                match = re.match(r".*Best Params: (.+), Test AUC: (.+)", line)
                if match:
                    best_params = eval(match.group(1))
                    test_auc = float(match.group(2))
                    row = {
                        "outer_fold": outer_fold,
                        "inner_fold": inner_fold,
                        "metric": "best_auc_TEST",
                        "value": test_auc,
                        **best_params  # 최적 하이퍼파라미터 추가
                    }
                    data.append(row)

    return pd.DataFrame(data)

# 로그 파일 파싱 실행
log_path = './log_FC.txt'  # 실제 로그 파일 경로로 변경 필요
parsed_df = parse_log_file(log_path)

# batch_size를 튜플로 변환하여 그룹화 가능하도록 설정
parsed_df['batch_size'] = parsed_df['batch_size'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# 'auc_TEST' 만 필터링
test_only_df = parsed_df[parsed_df['metric'] == 'auc_val']

# for test_df in test_only_df.groupby(['outer_fold', 'inner_fold', 'lam', 'd_dim', 'metric', 'nz', 'batch_size', 'lr', 'lr_critic']):
#     test_df = test_df[1]
#     print(test_df.iloc[1])
#     # 증가 추세 시각화
#     plt.figure(figsize=(10, 6))
#     plt.plot(test_df.index, test_df['value'], marker='o', linestyle='-', label='AUC_TEST Value')
#     plt.xlabel('Row Index')
#     plt.ylabel('AUC Value')
#     plt.title('Trend of AUC_TEST Values Over Rows')
#     plt.grid(True)
#     plt.legend()
#     plt.show()


In [65]:
import pandas as pd
import re

def parse_log_file(log_path):
    data = []
    fold = None
    hyperparams = {}
    capturing_hyperparams = False

    with open(log_path, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith("[Outer Fold"):
                # Outer Fold 시작 감지
                match = re.match(r"\[Outer Fold (\d+)\]", line)
                if match:
                    fold = int(match.group(1))
            
            elif line.startswith("Hyperparameters:"):
                # 하이퍼파라미터 시작 감지
                hyperparams = {}
                capturing_hyperparams = True

            elif capturing_hyperparams and ": " in line:
                # 하이퍼파라미터 추출
                if line.startswith("EN_model"):
                    capturing_hyperparams = False  # 모델 부분을 건너뛰기
                    continue
                
                key, value = map(str.strip, line.split(": ", 1))
                if key == "batch_size":
                    value = eval(value)  # 문자열 리스트를 실제 리스트로 변환
                elif re.match(r"^\d+(\.\d+)?$", value):  # 숫자 값 검증
                    value = float(value) if "." in value else int(value)
                
                # 특정 불필요한 키 제외
                if key not in ["TX_drug_feat_data_test", "TX_drug_adj_data_test", "TX_gexpr_data_test"]:
                    hyperparams[key] = value

            elif "-auc_val" in line or "-auc_TEST" in line:
                # AUC 결과 파싱
                result, metric = map(str.strip, line.split(" -"))
                row = {
                    "fold": fold,
                    "metric": metric,
                    "value": float(result),
                    **hyperparams  # 하이퍼파라미터 추가
                }
                data.append(row)
            
            elif line.startswith("Best Params"):
                # 최적 파라미터 및 AUC 추출
                match = re.match(r".*Best Params: (.+), Test AUC: (.+)", line)
                if match:
                    best_params = eval(match.group(1))
                    test_auc = float(match.group(2))
                    row = {
                        "fold": fold,
                        "metric": "best_auc_TEST",
                        "value": test_auc,
                        **best_params  # 최적 파라미터 추가
                    }
                    data.append(row)

    return pd.DataFrame(data)

# 로그 파일 파싱 실행
# log_path = './log_FC.txt'  # 실제 로그 파일 경로로 변경 필요
log_path = './log_FC_Lnz.txt'  # 실제 로그 파일 경로로 변경 필요
parsed_df = parse_log_file(log_path)

# batch_size를 튜플로 변환하여 그룹화 가능하도록 설정
parsed_df['batch_size'] = parsed_df['batch_size'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# 'auc_TEST' 만 필터링
test_only_df = parsed_df[parsed_df['metric'] == 'auc_TEST']

# 'auc_TEST' 기준으로 최고값과 해당 하이퍼파라미터 찾기
max_values = test_only_df.loc[
    test_only_df.groupby(['fold', 'nz', 'd_dim', 'lr', 'lr_critic', 'lam', 'batch_size'])['value'].idxmax()
]

# 결과를 CSV 파일로 저장
max_values.to_csv("max_hyperparameter_values_auc_TEST.csv", index=False)

# 결과 출력
print(max_values)


       fold    metric     value   nz  d_dim      lr  lr_critic   lam  \
22631     0  auc_TEST  0.823292  384    100  0.0010       0.01  0.10   
7397      0  auc_TEST  0.825449  384    100  0.0010       0.10  0.10   
761       0  auc_TEST  0.821265  384    100  0.0010       1.00  0.10   
27113     0  auc_TEST  0.820807  384    128  0.0001       0.10  1.00   
23827     0  auc_TEST  0.815736  384    128  0.0001       1.00  0.10   
9325      0  auc_TEST  0.816830  384    128  0.0001       1.00  1.00   
30231     0  auc_TEST  0.826303  384    128  0.0010       0.01  1.00   
31355     0  auc_TEST  0.820438  384    256  0.0001       0.10  1.00   
12967     0  auc_TEST  0.825913  512    100  0.0001       0.01  0.10   
28985     0  auc_TEST  0.821779  512    100  0.0010       0.10  0.01   
18439     0  auc_TEST  0.821719  512    100  0.0010       0.10  1.00   
6655      0  auc_TEST  0.823945  512    100  0.0010       1.00  0.10   
5281      0  auc_TEST  0.820400  512    128  0.0001       0.10  

In [61]:
parsed_df

,fold,metric,value,nz,d_dim,lr,lr_critic,lam,batch_size
0,0,auc_val,0.607140,100,256,0.0010,0.1,0.01,"(256, 28)"
1,0,auc_TEST,0.612100,100,256,0.0010,0.1,0.01,"(256, 28)"
2,0,auc_val,0.644117,100,256,0.0010,0.1,0.01,"(256, 28)"
3,0,auc_TEST,0.654644,100,256,0.0010,0.1,0.01,"(256, 28)"
4,0,auc_val,0.649007,100,256,0.0010,0.1,0.01,"(256, 28)"
...,...,...,...,...,...,...,...,...,...
41803,0,auc_TEST,0.790911,100,100,0.0001,1.0,1.00,"(128, 14)"
41804,0,auc_val,0.795022,100,100,0.0001,1.0,1.00,"(128, 14)"
41805,0,auc_TEST,0.792710,100,100,0.0001,1.0,1.00,"(128, 14)"
41806,0,auc_val,0.787634,100,100,0.0001,1.0,1.00,"(128, 14)"


In [66]:
test_only_df.groupby(['fold', 'nz', 'd_dim', 'lr', 'lr_critic', 'lam', 'batch_size'])['value']

for test_df in test_only_df.groupby(['fold', 'nz', 'd_dim', 'lr', 'lr_critic', 'lam', 'batch_size']):
    print(len(test_df[1]),test_df[1].iloc[1]['nz'])

638 384
615 384
624 384
1246 384
1358 384
1033 384
643 384
956 384
1033 512
579 512
601 512
634 512
1046 512
1086 512
1173 512
1221 512
1157 512
556 512


In [68]:
# Filter for 'auc_TEST' metric only
test_only_df = parsed_df[parsed_df['metric'] == 'auc_TEST']

# Find max values and corresponding hyperparameters for 'auc_TEST'
max_values = test_only_df.loc[test_only_df.groupby(['fold'])['value'].idxmax()]

# Save the result to a CSV file
max_values.to_csv("max_hyperparameter_values_auc_TEST.csv", index=False)

# Display the result
print(max_values)


       fold    metric     value   nz  d_dim     lr  lr_critic  lam batch_size
30231     0  auc_TEST  0.826303  384    128  0.001       0.01  1.0  (256, 28)


In [69]:

# Filter for 'auc_TEST' metric only
test_only_df = parsed_df[parsed_df['metric'] == 'auc_val']

# Find max values and corresponding hyperparameters for 'auc_TEST'
max_values = test_only_df.loc[test_only_df.groupby(['fold'])['value'].idxmax()]

# Save the result to a CSV file
max_values.to_csv("max_hyperparameter_values_auc_TEST.csv", index=False)

# Display the result
print(max_values)


       fold   metric     value   nz  d_dim      lr  lr_critic  lam batch_size
27980     0  auc_val  0.839922  384    128  0.0001        0.1  1.0  (128, 14)


In [12]:
len(max_values)

1